In [3]:

#  Import necessary modules
import random
import json
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

# Helper Function: Generate simulated sensor data

def generate_sensor_data():
    """Simulate reading from a temperature/humidity sensor."""
    temperature = round(random.uniform(15.0, 35.0), 1)  # Realistic range: 15–35°C
    humidity = round(random.uniform(30.0, 80.0), 1)     # Realistic range: 30–80%
    return {
        "temperature": temperature,
        "humidity": humidity
    }


# Encryption & Decryption Setup
# AES requires a 16-byte (128-bit) key for AES-128
KEY = get_random_bytes(16)  # In real IoT, this key would be pre-shared securely
IV = get_random_bytes(16)   # Initialization Vector (must be unique per message)

# Create cipher object for encryption
cipher_encrypt = AES.new(KEY, AES.MODE_CBC, IV)


#  Simulate IoT Device (Encrypt Sensor Data)

print("=== IoT Device (Sender) ===")
sensor_data = generate_sensor_data()
print(f"Original sensor data:")
print(f"  Temperature: {sensor_data['temperature']} C")
print(f"  Humidity: {sensor_data['humidity']} %")

# Convert data to string (JSON format for structure)
plain_text = json.dumps(sensor_data)
print(f"\nPlain text: {plain_text}")

# AES requires data length to be multiple of 16 bytes -> apply padding
padded_data = pad(plain_text.encode('utf-8'), AES.block_size)

# Encrypt the data
encrypted_data = cipher_encrypt.encrypt(padded_data)
print(f"\nEncrypted data (hex): {encrypted_data.hex()}")


# Simulate Server (Decrypt the Data)

print("\n\n=== Base Station (Receiver / Server) ===")
print("Simulating reception of encrypted data...")

# Recreate cipher for decryption (same KEY and IV)
cipher_decrypt = AES.new(KEY, AES.MODE_CBC, IV)

# Decrypt
decrypted_padded = cipher_decrypt.decrypt(encrypted_data)

# Remove padding
try:
    decrypted_bytes = unpad(decrypted_padded, AES.block_size)
    decrypted_text = decrypted_bytes.decode('utf-8')
    decrypted_data = json.loads(decrypted_text)

    print("Decrypted data:")
    print(f"  Temperature: {decrypted_data['temperature']} C")
    print(f"  Humidity: {decrypted_data['humidity']} %")

    # Verify integrity
    if sensor_data == decrypted_data:
        print("\nVerification successful: Original data matches received data.")
    else:
        print("\nError: Data mismatch after decryption.")

except (ValueError, KeyError) as e:
    print(f"\nDecryption failed: {e}")

=== IoT Device (Sender) ===
Original sensor data:
  Temperature: 32.2 C
  Humidity: 71.1 %

Plain text: {"temperature": 32.2, "humidity": 71.1}

Encrypted data (hex): fece2311d581ce23ca3f46f5839a4e27904dcf41f1458d494b8401281418073b8245039285dae9788a67c6850893af6f


=== Base Station (Receiver / Server) ===
Simulating reception of encrypted data...
Decrypted data:
  Temperature: 32.2 C
  Humidity: 71.1 %

Verification successful: Original data matches received data.
